In [1]:
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.cluster import AffinityPropagation
from sklearn.metrics.cluster import completeness_score
import numpy as np
import colorsys
import os
import cv2
import geoplotlib
import pandas as pd
import pyglet
from sklearn.utils.linear_assignment_ import linear_assignment
import time

np.set_printoptions(precision=4,linewidth=1000,suppress=True)
from math import *
def terrace_H():
    H0 = np.mat([[-1.6688907435, -6.9502305710, 940.69592392565],  # terrace1-c0
                 [1.1984806153, -10.7495778320, 868.29873467315],
                 [0.0004069210, -0.0209324057, 0.42949125235]])

    H1 = np.mat([[0.6174778372, -0.4836875683, 147.00510919005],  # terrace2-c1
                 [0.5798503075, 3.8204849039, -386.096405131],
                 [0.0000000001, 0.0077222239, -0.01593391935]])

    H2 = np.mat([[-0.2717592338, 1.0286363982, -17.6643219215],  # terrace2-c2
                 [-0.1373600672, -0.3326731339, 161.0109069274],
                 [0.0000600052, 0.0030858398, -0.04195162855]])

    H3 = np.mat([[-0.3286861858, 0.1142963200, 130.25528281945],  # terrace2-c3
                 [0.1809954834, -0.2059386455, 125.0260427323],
                 [0.0000693641, 0.0040168154, -0.08284534995]])
    H = [H0, H1, H2, H3]
    return H

def img_to_world(img_Points, H):
    input = np.mat([img_Points[0], img_Points[1], 1])
#    input = np.mat([img_Points[0][0][0], img_Points[0][0][1], 1])
    world_points = H * input.T
    world_points = world_points/world_points[2]
    return world_points
# detection_file = "/Users/chaojunli/Desktop/deep_sort/bbox_feats/bbox_feat_c1_c0.npy"
# features = []
# detections = np.load(detection_file)
# for detection in detections:
#     features.append(detection[6:])
# X = np.array(features)
# kmeans=KMeans(n_clusters=9)
# y_pred = kmeans.fit_predict(X)
# plt.scatter(X[:, 0], X[:, 1], c=y_pred)
# plt.show()
# feature_index = []
# feature_arr=[]
# feature_directory = "/Users/chaojunli/Desktop/deep_sort/s1c0/"
# for i in range(9):
#     feature_file = feature_directory + str(i)
#     for file in os.listdir(feature_file):
#         feature = np.load(feature_file+"/"+file)
#         feature_index.append(i)
#         feature_arr.append(feature[0])
# X = np.array(feature_arr)
# pca = PCA(n_components=15)
# X =  pca.fit_transform(X)
# kmeans=KMeans(n_clusters=9)
# # y_pred =  AffinityPropagation(preference=-50).fit(X)
# # labels =  AffinityPropagation(preference=-50).predict(X)
# y_pred =  kmeans.fit(X)
# labels =  kmeans.predict(X)
# plt.scatter(X[:, 0], X[:, 1], c=labels)
# print (completeness_score(feature_index,labels))

def create_unique_color_float(tag, hue_step=0.41):
    """Create a unique RGB color code for a given track id (tag).

    The color code is generated in HSV color space by moving along the
    hue angle and gradually changing the saturation.

    Parameters
    ----------
    tag : int
        The unique target identifying tag.
    hue_step : float
        Difference between two neighboring color codes in HSV space (more
        specifically, the distance in hue channel).

    Returns
    -------
    (float, float, float)
        RGB color code in range [0, 1]

    """
    h, v = (tag * hue_step) % 1, 1. - (int(tag * hue_step) % 4) / 5.
    r, g, b = colorsys.hsv_to_rgb(h, 1., v)
    return r, g, b

def create_unique_color_uchar(tag, hue_step=0.41):
    """Create a unique RGB color code for a given track id (tag).

    The color code is generated in HSV color space by moving along the
    hue angle and gradually changing the saturation.

    Parameters
    ----------
    tag : int
        The unique target identifying tag.
    hue_step : float
        Difference between two neighboring color codes in HSV space (more
        specifically, the distance in hue channel).

    Returns
    -------
    (int, int, int)
        RGB color code in range [0, 255]

    """
    r, g, b = create_unique_color_float(tag, hue_step)
    return int(255*r), int(255*g), int(255*b)

def rectangle(x, y, w, h,color,image,label=None):
        """Draw a rectangle.

        Parameters
        ----------
        x : float | int
            Top left corner of the rectangle (x-axis).
        y : float | int
            Top let corner of the rectangle (y-axis).
        w : float | int
            Width of the rectangle.
        h : float | int
            Height of the rectangle.
        label : Optional[str]
            A text label that is placed at the top left corner of the
            rectangle.

        """
        pt1 = int(x), int(y)
        pt2 = int(x + w), int(y + h)
        cv2.rectangle(image, pt1, pt2, color, 2)
        if label is not None:
            text_size = cv2.getTextSize(
                label, cv2.FONT_HERSHEY_PLAIN, 1, 2)

            center = pt1[0] + 5, pt1[1] + 5 + text_size[0][1]
            pt2 = pt1[0] + 10 + text_size[0][0], pt1[1] + 10 + \
                text_size[0][1]
            cv2.rectangle(image, pt1, pt2, color, -1)
            cv2.putText(image, label, center, cv2.FONT_HERSHEY_PLAIN,
                        1, (255, 255, 255), 2)


feature_index = {}
feature_arr={}
bbox_arr = {}
frameIndex_arr = {}
video_player = {}
file = {}
coordinates = {}
X = []
output_directory = "/Users/chaojunli/Desktop/deep-sort-tracking/multitracking_output.mp4"
feature_directory = "/Users/chaojunli/Desktop/deep-sort-tracking/real2/feats_pcb/test2.npy,/Users/chaojunli/Desktop/deep-sort-tracking/real2/feats_pcb/test1.npy"
video_directory = "/Users/chaojunli/Desktop/deep-sort-tracking/real2/test2.MOV,/Users/chaojunli/Desktop/deep-sort-tracking/real2/test1.mp4"
feature_directory = feature_directory.split(",")
video_directory = video_directory.split(",")
for i in range(len(video_directory)):
    video_player[i] = cv2.VideoCapture(video_directory[i])
    file[i] = np.load(feature_directory[i])
    bbox_arr[i] = []
    feature_index[i] = []
    feature_arr[i] = []
    frameIndex_arr[i] = []
    coordinates[i] = []
fourcc = cv2.VideoWriter_fourcc(*'MP4V') 
out = cv2.VideoWriter(output_directory,fourcc, 20.0, (1280,480))
for i in range(len(file)):
    for row in file[i]:
#         if row[4] < 290:
#             print ("i is {}, height is {}, width is {}, bounding box is {}".format(i,row[4],row[3],row[1:5]))
#             continue
        if int(row[0]) > 1800:
            break
        frameIndex_arr[i].append(row[0])
        bbox_arr[i].append(row[1:5])
        feature_arr[i].append(row[6:])
        X.append(row[6:])
X = np.array(X)
pca = PCA(n_components=15)
X =  pca.fit_transform(X)
# for i in range(len(feature_arr)):
#     feature_arr[i] = list(pca.fit_transform(np.array(feature_arr[i])))
split_size = len(feature_arr[0])
feature_arr[0] = list(X[:split_size])
feature_arr[1] = list(X[split_size:])
kmeans=KMeans(n_clusters=5)  
y_pred =  kmeans.fit(X)
labels =  kmeans.predict(X)
centers = kmeans.cluster_centers_
# af = AffinityPropagation(preference=-50).fit(X)
# centers = af.cluster_centers_indices_
for k in range(len(video_directory)):
    for i in range(len(centers)):
        coordinate = []
        coordinate.append(-300)
        for j in range(6):
            coordinate.append(0)
        coordinates[k].append(coordinate)
index = 1
# while index <= frameIndex_arr[-1]:
while index <= 1800:
    for k in range(len(video_player)):
        (grabbed, frame) = video_player[k].read() 
        print ("processing frame index:{}".format(index))
        image = frame.copy()
        detection_arr = []
        detection_indices = [i for i, e in enumerate(frameIndex_arr[k]) if e == index]
        cost_matrix = np.zeros((len(centers),len(detection_indices)))
        world_coordinate_arr = []
        for j in range(len(detection_indices)):
            bbox = bbox_arr[k][detection_indices[j]]
            coordinate = [bbox[0]+bbox[2]/2, bbox[1]+bbox[3]]
            world_coordinate = np.array(coordinate, dtype=np.float32)
            world_coordinate = img_to_world(world_coordinate, terrace_H()[0])
            world_coordinate_arr.append(world_coordinate[0:2])
        for i in range(len(centers)):
            for j in range(len(detection_indices)):
                diff_element = [x-y for x, y in zip(centers[i],feature_arr[k][detection_indices[j]])]
                cost_matrix[i,j] = sum(x**2 for x in diff_element)
        indices = linear_assignment(cost_matrix)
#         print ("indices is {}".format(indices))
        matches, unmatched_detections = [], []
        for row, col in indices:
            detection_idx = detection_indices[col]
            matches.append((row, detection_idx))
#             print ("k is {}, row is {}".format(k,row))
            coordinates[k][row][0] = index
            coordinates[k][row][1:3] = world_coordinate_arr[col][0:2]
            coordinates[k][row][3:] = bbox_arr[k][detection_idx]
        for i in range(len(coordinates[k])):
            if coordinates[k][i][0] != index:
                continue
            bbox = coordinates[k][i][3:]
#             bbox[1] -= 25
#             bbox[3] += 50
            color = create_unique_color_uchar(i+1)
            rectangle(*bbox,color,image,label=str(i+1))        
        image = cv2.resize(image,(640,480))
        if k > 0:
            image_concate = np.concatenate((image_concate,image),axis=1)
        else:
            image_concate = image
    index += 1
    remaining_time = 1
    out.write(image_concate)
    cv2.imshow("camera",image_concate)
#     time.sleep(0.02)
    key = cv2.waitKey(remaining_time)
out.release()
cv2.destroyAllWindows()


/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


processing frame index:1
processing frame index:1
processing frame index:2
processing frame index:2
processing frame index:3
processing frame index:3
processing frame index:4
processing frame index:4
processing frame index:5
processing frame index:5
processing frame index:6
processing frame index:6
processing frame index:7
processing frame index:7
processing frame index:8
processing frame index:8
processing frame index:9
processing frame index:9
processing frame index:10
processing frame index:10
processing frame index:11
processing frame index:11
processing frame index:12
processing frame index:12
processing frame index:13
processing frame index:13
processing frame index:14
processing frame index:14
processing frame index:15
processing frame index:15
processing frame index:16
processing frame index:16
processing frame index:17
processing frame index:17
processing frame index:18
processing frame index:18
processing frame index:19
processing frame index:19
processing frame index:20
proc

processing frame index:159
processing frame index:159
processing frame index:160
processing frame index:160
processing frame index:161
processing frame index:161
processing frame index:162
processing frame index:162
processing frame index:163
processing frame index:163
processing frame index:164
processing frame index:164
processing frame index:165
processing frame index:165
processing frame index:166
processing frame index:166
processing frame index:167
processing frame index:167
processing frame index:168
processing frame index:168
processing frame index:169
processing frame index:169
processing frame index:170
processing frame index:170
processing frame index:171
processing frame index:171
processing frame index:172
processing frame index:172
processing frame index:173
processing frame index:173
processing frame index:174
processing frame index:174
processing frame index:175
processing frame index:175
processing frame index:176
processing frame index:176
processing frame index:177
p

processing frame index:314
processing frame index:314
processing frame index:315
processing frame index:315
processing frame index:316
processing frame index:316
processing frame index:317
processing frame index:317
processing frame index:318
processing frame index:318
processing frame index:319
processing frame index:319
processing frame index:320
processing frame index:320
processing frame index:321
processing frame index:321
processing frame index:322
processing frame index:322
processing frame index:323
processing frame index:323
processing frame index:324
processing frame index:324
processing frame index:325
processing frame index:325
processing frame index:326
processing frame index:326
processing frame index:327
processing frame index:327
processing frame index:328
processing frame index:328
processing frame index:329
processing frame index:329
processing frame index:330
processing frame index:330
processing frame index:331
processing frame index:331
processing frame index:332
p

processing frame index:467
processing frame index:467
processing frame index:468
processing frame index:468
processing frame index:469
processing frame index:469
processing frame index:470
processing frame index:470
processing frame index:471
processing frame index:471
processing frame index:472
processing frame index:472
processing frame index:473
processing frame index:473
processing frame index:474
processing frame index:474
processing frame index:475
processing frame index:475
processing frame index:476
processing frame index:476
processing frame index:477
processing frame index:477
processing frame index:478
processing frame index:478
processing frame index:479
processing frame index:479
processing frame index:480
processing frame index:480
processing frame index:481
processing frame index:481
processing frame index:482
processing frame index:482
processing frame index:483
processing frame index:483
processing frame index:484
processing frame index:484
processing frame index:485
p

...

processing frame index:1080
processing frame index:1081
processing frame index:1081
processing frame index:1082
processing frame index:1082
processing frame index:1083
processing frame index:1083
processing frame index:1084
processing frame index:1084
processing frame index:1085
processing frame index:1085
processing frame index:1086
processing frame index:1086
processing frame index:1087
processing frame index:1087
processing frame index:1088
processing frame index:1088
processing frame index:1089
processing frame index:1089
processing frame index:1090
processing frame index:1090
processing frame index:1091
processing frame index:1091
processing frame index:1092
processing frame index:1092
processing frame index:1093
processing frame index:1093
processing frame index:1094
processing frame index:1094
processing frame index:1095
processing frame index:1095
processing frame index:1096
processing frame index:1096
processing frame index:1097
processing frame index:1097
processing frame ind

processing frame index:1230
processing frame index:1230
processing frame index:1231
processing frame index:1231
processing frame index:1232
processing frame index:1232
processing frame index:1233
processing frame index:1233
processing frame index:1234
processing frame index:1234
processing frame index:1235
processing frame index:1235
processing frame index:1236
processing frame index:1236
processing frame index:1237
processing frame index:1237
processing frame index:1238
processing frame index:1238
processing frame index:1239
processing frame index:1239
processing frame index:1240
processing frame index:1240
processing frame index:1241
processing frame index:1241
processing frame index:1242
processing frame index:1242
processing frame index:1243
processing frame index:1243
processing frame index:1244
processing frame index:1244
processing frame index:1245
processing frame index:1245
processing frame index:1246
processing frame index:1246
processing frame index:1247
processing frame ind

processing frame index:1380
processing frame index:1380
processing frame index:1381
processing frame index:1381
processing frame index:1382
processing frame index:1382
processing frame index:1383
processing frame index:1383
processing frame index:1384
processing frame index:1384
processing frame index:1385
processing frame index:1385
processing frame index:1386
processing frame index:1386
processing frame index:1387
processing frame index:1387
processing frame index:1388
processing frame index:1388
processing frame index:1389
processing frame index:1389
processing frame index:1390
processing frame index:1390
processing frame index:1391
processing frame index:1391
processing frame index:1392
processing frame index:1392
processing frame index:1393
processing frame index:1393
processing frame index:1394
processing frame index:1394
processing frame index:1395
processing frame index:1395
processing frame index:1396
processing frame index:1396
processing frame index:1397
processing frame ind

processing frame index:1529
processing frame index:1529
processing frame index:1530
processing frame index:1530
processing frame index:1531
processing frame index:1531
processing frame index:1532
processing frame index:1532
processing frame index:1533
processing frame index:1533
processing frame index:1534
processing frame index:1534
processing frame index:1535
processing frame index:1535
processing frame index:1536
processing frame index:1536
processing frame index:1537
processing frame index:1537
processing frame index:1538
processing frame index:1538
processing frame index:1539
processing frame index:1539
processing frame index:1540
processing frame index:1540
processing frame index:1541
processing frame index:1541
processing frame index:1542
processing frame index:1542
processing frame index:1543
processing frame index:1543
processing frame index:1544
processing frame index:1544
processing frame index:1545
processing frame index:1545
processing frame index:1546
processing frame ind

processing frame index:1677
processing frame index:1677
processing frame index:1678
processing frame index:1678
processing frame index:1679
processing frame index:1679
processing frame index:1680
processing frame index:1680
processing frame index:1681
processing frame index:1681
processing frame index:1682
processing frame index:1682
processing frame index:1683
processing frame index:1683
processing frame index:1684
processing frame index:1684
processing frame index:1685
processing frame index:1685
processing frame index:1686
processing frame index:1686
processing frame index:1687
processing frame index:1687
processing frame index:1688
processing frame index:1688
processing frame index:1689
processing frame index:1689
processing frame index:1690
processing frame index:1690
processing frame index:1691
processing frame index:1691
processing frame index:1692
processing frame index:1692
processing frame index:1693
processing frame index:1693
processing frame index:1694
processing frame ind

In [2]:
ratio = pca.explained_variance_ratio_
print(np.sum(ratio))

0.47032645674840373
